In [1]:
import pandas as pd
data = pd.read_csv('../dataset_customer_churn.csv', sep='^')
data.drop(labels=['A006_REGISTRO_ANS', 'CODIGO_BENEFICIARIO', 'CLIENTE', 'CD_USUARIO', 'CODIGO_FORMA_PGTO_MENSALIDADE', 'A006_NM_PLANO', 'DIAS_ATE_REALIZAR_ALTO_CUSTO', 'CD_ASSOCIADO', 'ESTADO_CIVIL'], axis=1, inplace=True)
is_NAN = data[data.isna().any(axis=1)]
data.drop(is_NAN.index, axis=0, inplace=True)
dict_replace = {
    "SIM": 1,
    "NAO": 0,
    'F': 0,
    'M': 1,
    'DESATIVADO': 1,
    'ATIVO': 0,

}
data.replace(dict_replace, inplace=True)
data.drop(labels=182212, axis=0, inplace=True)
data_dummified = pd.get_dummies(data)

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split

x = data_dummified.drop(labels=['SITUACAO'], axis=1)
y = data_dummified['SITUACAO']
np.random.seed(1010)
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, stratify=y)

In [5]:
from sklearn.neural_network import MLPClassifier
mlp_classifier = MLPClassifier()
mlp_classifier.fit(train_x,train_y)
predicts_mlp = mlp_classifier.predict(test_x)

In [6]:
from sklearn.metrics import f1_score,recall_score,accuracy_score,precision_score
def all_scores(predicts, test_y):
    print(f"Accuracy Score: {accuracy_score(test_y,predicts):.3f}\nRecall Score: {recall_score(test_y,predicts):.3f}\nPrecision Score: {precision_score(test_y,predicts):.3f}\nF1 Score: {f1_score(test_y,predicts):.3f}\n")
def scores(validation_results):
    media = validation_results['test_score'].mean()
    desvio_padrao = validation_results['test_score'].std()
    print("Accuracy médio {:.2f}".format(media))
    print("Intervalo [{:.2f}, {:.2f}]".format((media - 2 * desvio_padrao), (media + 2 * desvio_padrao)))

In [7]:
all_scores(predicts_mlp, test_y)

Accuracy Score: 0.859
Recall Score: 0.914
Precision Score: 0.841
F1 Score: 0.876



In [8]:
from sklearn.model_selection import cross_validate, KFold

validation_results = cross_validate(mlp_classifier,x,y,cv=KFold(n_splits=5, shuffle=True))
scores(validation_results)
validation_results_df = pd.DataFrame(validation_results)
validation_results_df

Accuracy médio 0.85
Intervalo [0.83, 0.87]


,fit_time,score_time,test_score
0,239.757891,0.076793,0.839385
1,116.304569,0.067819,0.838453
2,152.074033,0.088844,0.834468
3,201.136656,0.081781,0.851678
4,647.134322,0.123186,0.861487
5,929.901983,0.214939,0.854758
6,193.558874,0.183028,0.845959
7,608.231486,0.112686,0.859701
8,160.082544,0.105720,0.848961
9,266.745826,0.124668,0.845001
